# **Customer Churn Prediction**

#### **What is customer churn?**  
Customer churn refers to the percentage of customers who stop using a company's product or service within a given time frame. This metric helps businesses gauge customer satisfaction and loyalty while also providing insights into potential revenue fluctuations.

Churn is especially critical for subscription-based businesses, such as SaaS companies, which rely on recurring revenue. Understanding churn patterns allows them to anticipate financial impact and take proactive measures.

Also known as customer attrition, churn is the opposite of customer retention, which focuses on maintaining long-term customer relationships. Reducing churn should be a key part of any customer engagement strategy, ensuring consistent interactions between businesses and their customers, whether online or in person.

A strong customer retention plan plays a crucial role in minimizing churn. Companies should track churn rates regularly to assess their risk of revenue loss and identify areas for improvement.

<br>

**Source:** IBM. Customer Churn. Retrieved from https://www.ibm.com/think/topics/customer-churn

---

Prepare the libraries.

In [69]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import shap

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

Prepare the dataset/s.

In [70]:
pd.pandas.set_option('display.max_columns',None)

train_path = '../data/CustomerChurnDataset/customer_churn_dataset-testing-master.csv'
test_path = '../data/CustomerChurnDataset/customer_churn_dataset-training-master.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

Since we saw in our EDA that we only have 1 row of missing value/s from test_df lets drop that row.  
Prepare the train and test datasets.

In [71]:
# drop missing values 
test_df.dropna(inplace=True)
train_df.dropna(inplace=True)

# drop customer id column
train_df = train_df.drop(columns=['CustomerID'])
test_df = test_df.drop(columns=['CustomerID'])

y_train = train_df['Churn']
x_train = train_df.drop(columns=['Churn'])

y_test = test_df['Churn']
x_test = test_df.drop(columns=['Churn'])

Get the numerical and categorical features.

In [72]:
categorical_features = x_train.select_dtypes(include=['object']).columns.tolist()
numerical_features = x_train.select_dtypes(include=['int64','float64']).columns.tolist()

print(categorical_features)
print(numerical_features)

['Gender', 'Subscription Type', 'Contract Length']
['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Total Spend', 'Last Interaction']


Prepare the preprocessor for the pipeline.

In [73]:
preprocessor = ColumnTransformer([
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
], remainder='passthrough')

Set up the hyperparameters for tuning.

In [74]:
params_space = {
    'eta': hp.uniform('eta', 0.01,0.2),
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'subsample': hp.uniform('subsample', 0.5,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 1000, 10),
    'seed': 0
}

def objective(params):
    # convert the quniforms into int since they are float
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['reg_alpha'] = int(params['reg_alpha'])
    
    pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('classifier', XGBClassifier(**params, eval_metric="logloss"))
    ])

    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    auc = roc_auc_score(y_test, y_pred) 
    print(f"Params: {params}, AUC-ROC: {auc:.4f}")

    return {'loss': -auc, 'status': STATUS_OK}

Run the hyperparameter tuning.


In [75]:
trials = Trials()

best_params = fmin(
    fn = objective,
    space = params_space,
    algo = tpe.suggest,
    max_evals = 100,
    trials = trials
)

print("\n✅ Best Hyperparameters Found:", best_params)

Params: {'colsample_bytree': 0.9630273106063283, 'eta': 0.1779679995060585, 'gamma': 1.1294052984814682, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 60, 'reg_alpha': 178, 'reg_lambda': 0.7985953935209215, 'seed': 0, 'subsample': 0.5014491785435579}, AUC-ROC: 0.6338
Params: {'colsample_bytree': 0.5099049671359164, 'eta': 0.01884491615219518, 'gamma': 5.060916966389359, 'max_depth': 7, 'min_child_weight': 9, 'n_estimators': 510, 'reg_alpha': 159, 'reg_lambda': 0.7907489672711489, 'seed': 0, 'subsample': 0.7824697222453074}, AUC-ROC: 0.6372
Params: {'colsample_bytree': 0.5907841273937582, 'eta': 0.08145581683707065, 'gamma': 2.298952003920321, 'max_depth': 17, 'min_child_weight': 0, 'n_estimators': 650, 'reg_alpha': 178, 'reg_lambda': 0.36856517524184673, 'seed': 0, 'subsample': 0.6397142051677264}, AUC-ROC: 0.6362
Params: {'colsample_bytree': 0.5813086486634221, 'eta': 0.12587983692993626, 'gamma': 1.866711592168687, 'max_depth': 14, 'min_child_weight': 1, 'n_estimators': 730,

Get the best hyperparameters.


In [76]:
int_params = {"max_depth", "n_estimators", "min_child_weight", "reg_alpha"}

def convert_params(params):
    return {k: int(v) if k in int_params else float(v) for k, v in params.items()}

best_params = convert_params(best_params)

Create the pipeline.

In [77]:
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', XGBClassifier(**best_params))
])

Train the model and make predictions.

In [78]:
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)
y_proba = pipeline.predict_proba(x_test) 

Evaluate the model.

In [79]:
def evaluate_model(test, pred, proba):
    auc_roc = roc_auc_score(test, proba[:, 1]) 
    
    print("XGBoost Classification Report:\n", classification_report(test, pred))
    # print("XGBoost AUC-ROC:", roc_auc_score(test, pred))
    print(f"XGBoost AUC-ROC: {auc_roc:.4f}")
    
evaluate_model(y_test, y_pred, y_proba)

XGBoost Classification Report:
               precision    recall  f1-score   support

         0.0       0.52      1.00      0.68    190833
         1.0       1.00      0.29      0.44    249999

    accuracy                           0.59    440832
   macro avg       0.76      0.64      0.56    440832
weighted avg       0.79      0.59      0.55    440832

XGBoost AUC-ROC: 0.8696
